In [137]:
%load_ext autoreload
%autoreload 2
%pylab inline
import json

import sys
import glob
import pandas as pda
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_br import BR_DIR, make_dqn_agent
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator, TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
import torch
from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from open_spiel.python import rl_environment
import yaml
from open_spiel.python.pytorch.dqn import DQN
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [138]:
# experiment_dir = '/shared/outputs/jan2_big_game_2/lstm_deep'
# checkpoint_name = 'checkpoint_3000000'
br_player = 0
report_freq = 5_000
game_name = 'small_game_2.json'

In [139]:
game = smart_load_sequential_game('clock_auction', dict(filename=game_name))
env = rl_environment.Environment(game)


In [140]:
game_config = json.load(open(f'/apps/open_spiel/configs/{game_name}', 'r'))

In [141]:
agent_1 = TakeSingleActionDecorator(StraightforwardAgent(1, game_config, game.num_distinct_actions()), game.num_distinct_actions())

In [172]:
with open('/apps/open_spiel/notebooks/configs/jan10mlp/mlp-lowlr.yml', 'r') as f:
    config = yaml.safe_load(f)
    config['rl_learning_rate'] = 0.01
config['double_dqn'] = False

In [173]:
agents = []
# agents.append(DQN(.
#                0,
#                len(game.new_initial_state().child(0).child(0).information_state_tensor()),
#                game.num_distinct_actions(),
#                learning_rate = 0.001
# ))
# from open_spiel.python.algorithms.tabular_qlearner import QLearner
# agents.append(QLearner(0, game.num_distinct_actions(), step_size=0.01))

agents.append(make_dqn_agent(0, config, env, game, game_config))
agents.append(agent_1)

In [174]:
episode_lengths = []
for i in range(1_000_000):
    time_step = env.reset()
    episode_length = 0
    if i % report_freq == 0:
        print(i)
#         print(agents[0]._q_values[str(time_step.observations["info_state"][0])])
        print(check_on_q_values(agents[0], game))
#         print(pd.Series(episode_lengths).describe())
        print()
    
    while not time_step.last():
        episode_length += 1

        
        player_id = time_step.observations["current_player"]
        agent = agents[player_id]
        agent_output = agent.step(time_step, is_evaluation=False)
        action_list = [agent_output.action]
        time_step = env.step(action_list)

    episode_lengths.append(episode_length)
    # Episode is over, step all agents with final info state.
    for player_id, agent in enumerate(agents):
        agent.step(time_step, is_evaluation=False)


0
{'Bid for 0,0 licenses @ $0 with activity 0': tensor(-4.5871), 'Bid for 0,1 licenses @ $50 with activity 50': tensor(24.1382), 'Bid for 1,0 licenses @ $75 with activity 75': tensor(-16.9303), 'Bid for 1,1 licenses @ $125 with activity 125': tensor(-52.1428)}

5000
{'Bid for 0,0 licenses @ $0 with activity 0': tensor(44531.5625), 'Bid for 0,1 licenses @ $50 with activity 50': tensor(-41710.6484), 'Bid for 1,0 licenses @ $75 with activity 75': tensor(2347.2915), 'Bid for 1,1 licenses @ $125 with activity 125': tensor(17988.2910)}

10000
{'Bid for 0,0 licenses @ $0 with activity 0': tensor(7509.7871), 'Bid for 0,1 licenses @ $50 with activity 50': tensor(-10174.2920), 'Bid for 1,0 licenses @ $75 with activity 75': tensor(1007.1894), 'Bid for 1,1 licenses @ $125 with activity 125': tensor(9149.8018)}

15000
{'Bid for 0,0 licenses @ $0 with activity 0': tensor(1085.3983), 'Bid for 0,1 licenses @ $50 with activity 50': tensor(-2239.2827), 'Bid for 1,0 licenses @ $75 with activity 75': tens

KeyboardInterrupt: 